# Connectie met de gecombineerde database

In [12]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlalchemy.types import Numeric, Integer, String


DB = {'servername': 'XPS15\\SQLEXPRESS', 'database': 'gecombineerde_database2'}

#SQLAlchemy engine voor SQL Server
engine_str = (
    f"mssql+pyodbc://{DB['servername']}/{DB['database']}?"
    "trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
)

engine = create_engine(engine_str)

#week3 database connectie

wk3_db = {'servername': 'XPS15\\SQLEXPRESS', 'database': 'week3_database'}
wk3_engine_str = (
    f"mssql+pyodbc://{wk3_db['servername']}/{wk3_db['database']}?"
    "trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
)
wk3_engine = create_engine(wk3_engine_str)

# Alles tot dataframe maken

In [13]:
# Laad alle tabellen in DataFrames
df_age_group = pd.read_sql("SELECT * FROM age_group", engine)
df_country = pd.read_sql("SELECT * FROM country", engine)
df_course = pd.read_sql("SELECT * FROM course", engine)
df_inventory = pd.read_sql("SELECT * FROM inventory", engine)
df_order_details = pd.read_sql("SELECT * FROM order_details", engine)
df_order_header = pd.read_sql("SELECT * FROM order_header", engine)
df_order_method = pd.read_sql("SELECT * FROM order_method", engine)
df_product = pd.read_sql("SELECT * FROM product", engine)
df_product_forecast = pd.read_sql("SELECT * FROM product_forecast", engine)
df_product_line = pd.read_sql("SELECT * FROM product_line", engine)
df_product_type = pd.read_sql("SELECT * FROM product_type", engine)
df_retailer = pd.read_sql("SELECT * FROM retailer", engine)
df_retailer_contact = pd.read_sql("SELECT * FROM retailer_contact", engine)
df_retailer_headquarters = pd.read_sql("SELECT * FROM retailer_headquarters", engine)
df_retailer_segment = pd.read_sql("SELECT * FROM retailer_segment", engine)
df_retailer_site = pd.read_sql("SELECT * FROM retailer_site", engine)
df_retailer_type = pd.read_sql("SELECT * FROM retailer_type", engine)
df_return_reason = pd.read_sql("SELECT * FROM return_reason", engine)
df_returned_item = pd.read_sql("SELECT * FROM returned_item", engine)
df_sales_branch = pd.read_sql("SELECT * FROM sales_branch", engine)
df_sales_demographic = pd.read_sql("SELECT * FROM sales_demographic", engine)
df_sales_staff = pd.read_sql("SELECT * FROM sales_staff", engine)
df_sales_territory = pd.read_sql("SELECT * FROM sales_territory", engine)
df_satisfaction = pd.read_sql("SELECT * FROM satisfaction", engine)
df_satisfaction_type = pd.read_sql("SELECT * FROM satisfaction_type", engine)
df_training = pd.read_sql("SELECT * FROM training", engine)


# dataframe mergen voor de dimensies en feiten tabllen en naar database sturen

## dimensie tabel1 dim_country

In [14]:
# Start met de basis country dataframe
dim_country_df = df_country.copy()

# Merge met sales_territory om TERRITORY_NAME toe te voegen
dim_country_df = dim_country_df.merge(
    df_sales_territory[['SALES_TERRITORY_CODE', 'TERRITORY_NAME_EN']],
    on='SALES_TERRITORY_CODE',
    how='left'
)

# Hernoem kolommen naar de gewenste namen
dim_country_df = dim_country_df.rename(columns={
    'COUNTRY': 'NAME',
    'CURRENCY_NAME': 'CURRENCY',
    'COUNTRY_CODE': 'COUNTRY_CODE',  # Blijft hetzelfde
    'FLAG_IMAGE': 'IMAGE',
    'TERRITORY_NAME_EN': 'TERRITORY_NAME'
})

# Selecteer de juiste kolommen in de juiste volgorde
dim_country_df = dim_country_df[['COUNTRY_CODE', 'NAME', 'LANGUAGE', 'IMAGE', 'CURRENCY', 'TERRITORY_NAME']]

dim_country_df

,COUNTRY_CODE,NAME,LANGUAGE,IMAGE,CURRENCY,TERRITORY_NAME
0,1,France,FR,F01,francs,Central Europe
1,2,Germany,DE,F02,marks,Central Europe
2,3,United States,EN,F03,dollars,Americas
3,4,Canada,EN,F04,dollars,Americas
4,5,Austria,DE,F05,schillings,Southern Europe
5,6,Italy,IT,F06,lira,Southern Europe
6,7,Netherlands,NL,F07,guilders,Northern Europe
7,8,Switzerland,DE,F08,francs,Central Europe
8,9,United Kingdom,EN,F09,pounds,Central Europe
9,10,Sweden,SV,F10,krona,Northern Europe


## dimensie tabel2 dim_product

In [15]:
dim_product_df = df_product.merge(
    df_product_type[['PRODUCT_TYPE_CODE', 'PRODUCT_TYPE_EN', 'PRODUCT_LINE_CODE']],
    on='PRODUCT_TYPE_CODE', how='left'
).merge(
    df_product_line[['PRODUCT_LINE_CODE', 'PRODUCT_LINE_EN']],
    on='PRODUCT_LINE_CODE', how='left'
)

# Hernoem de kolommen zodat ze overeenkomen met de specificatie:
dim_product_df = dim_product_df.rename(columns={
    'PRODUCT_TYPE_EN': 'PRODUCTION_TYPE',
    'PRODUCT_LINE_EN': 'PRODUCT_LINE'
})

# Converteer INTRODUCTION_DATE naar datetime en formatteer als een integer in het formaat YYYYMMDD
dim_product_df['INTRODUCTION_DATE'] = pd.to_datetime(dim_product_df['INTRODUCTION_DATE']) \
   .dt.strftime('%Y%m%d').astype(int)

# Selecteer de  kolommen:
dim_product_df = dim_product_df[['PRODUCT_NUMBER', 'PRODUCT_NAME', 'DESCRIPTION', 'INTRODUCTION_DATE',
                                 'PRODUCTION_COST', 'MARGIN', 'PRODUCT_IMAGE', 'PRODUCTION_TYPE',
                                 'PRODUCT_LINE']]

dim_product_df


,PRODUCT_NUMBER,PRODUCT_NAME,DESCRIPTION,INTRODUCTION_DATE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,PRODUCTION_TYPE,PRODUCT_LINE
0,1,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",19950215,4.00,0.33,P01CE1CG1.jpg,Cooking Gear,Camping Equipment
1,2,TrailChef Canteen,Aluminum canteen. Rugged fleece-lined cover wi...,19950215,9.22,0.23,P02CE1CG1.jpg,Cooking Gear,Camping Equipment
2,3,TrailChef Kitchen Kit,"Zippered nylon pouch contains cutlery for two,...",19950215,15.93,0.28,P03CE1CG1.jpg,Cooking Gear,Camping Equipment
3,4,TrailChef Cup,Tin cup. Holds 0.4 liters. Weight: 60 g,19950215,5.00,0.28,P04CE1CG1.jpg,Cooking Gear,Camping Equipment
4,5,TrailChef Cook Set,All you will ever need on the trail. Pot grip...,19950215,34.97,0.30,P05CE1CG1.jpg,Cooking Gear,Camping Equipment
...,...,...,...,...,...,...,...,...,...
110,111,Blue Steel Max Putter,Putter head is composed from a single piece of...,20031215,81.80,0.55,P111GE5PT20.jpg,Putters,Golf Equipment
111,112,Course Pro Golf and Tee Set,Set includes two premium golf ball and an asso...,20040110,6.00,0.33,P112GE5GA21.jpg,Golf Accessories,Golf Equipment
112,113,Course Pro Umbrella,Large red and white patterned golf umbrella.,20040115,6.00,0.50,P113GE5GA21.jpg,Golf Accessories,Golf Equipment
113,114,Course Pro Golf Bag,High quality golf bag that includes carrying s...,20031215,80.00,0.60,P114GE5GA21.jpg,Golf Accessories,Golf Equipment


## dimensie tabel3 dim_retailer

In [16]:
# Begin met df_retailer als basis
dim_retailer_df = df_retailer.copy()

# Merge met retailer_headquarters op RETAILER_CODEMR
dim_retailer_df = dim_retailer_df.merge(df_retailer_headquarters, on='RETAILER_CODEMR', how='outer')

# Merge met retailer_segment op SEGMENT_CODE
dim_retailer_df = dim_retailer_df.merge(df_retailer_segment, on='SEGMENT_CODE', how='outer')

# Merge met retailer_site op RETAILER_CODE
dim_retailer_df = dim_retailer_df.merge(df_retailer_site, on='RETAILER_CODE', how='left')

# Merge met retailer_contact op RETAILER_SITE_CODE en hernoem de contactkolommen
dim_retailer_df = dim_retailer_df.merge(
    df_retailer_contact.rename(columns={
        'FIRST_NAME': 'CONTACT_FIRST_NAME',
        'LAST_NAME': 'CONTACT_LAST_NAME',
        'E_MAIL': 'CONTACT_E_MAIL',
        'JOB_POSITION_EN': 'CONTACT_JOB_POSITION_EN'
    }),
    on='RETAILER_SITE_CODE',
    how='outer'
)

# Merge met retailer_type op RETAILER_TYPE_CODE met how='outer'
dim_retailer_df = dim_retailer_df.merge(df_retailer_type, on='RETAILER_TYPE_CODE', how='outer')

# Hernoem dubbele kolommen naar de gewenste namen
dim_retailer_df = dim_retailer_df.rename(columns={
    'ADDRESS1_x': 'ADDRESS_1',
    'ADDRESS2_x': 'ADDRESS_2',
    'CITY_x': 'CITY',
    'REGION_x': 'REGION',
    'COUNTRY_CODE_x': 'COUNTRY_CODE',
    'FAX': 'CONTACT_FAX'
})

# Selecteer de gewenste kolommen in de juiste volgorde
final_cols = [
    'RETAILER_CODE',
    'RETAILER_SITE_CODE',
    'COMPANY_NAME',
    'RETAILER_NAME',
    'ADDRESS_1',
    'ADDRESS_2',
    'CITY',
    'REGION',
    'COUNTRY_CODE',
    'SEGMENT_NAME',
    'SEGMENT_DESCRIPTION',
    'RETAILER_TYPE_EN',
    'CONTACT_FIRST_NAME',
    'CONTACT_LAST_NAME',
    'CONTACT_E_MAIL',
    'CONTACT_JOB_POSITION_EN',
    'CONTACT_FAX',
    'CONTACT_GENDER',
    'CONTACT_EXTENSION'
]

dim_retailer_df = dim_retailer_df[[col for col in final_cols if col in dim_retailer_df.columns]]

dim_retailer_df = dim_retailer_df.drop_duplicates(subset=['RETAILER_CODE'])
dim_retailer_df = dim_retailer_df.dropna(subset=['RETAILER_CODE'])
dim_retailer_df = dim_retailer_df.drop_duplicates(subset=['RETAILER_SITE_CODE'])
dim_retailer_df = dim_retailer_df.dropna(subset=['RETAILER_SITE_CODE'])
dim_retailer_df

,RETAILER_CODE,RETAILER_SITE_CODE,COMPANY_NAME,RETAILER_NAME,ADDRESS_1,ADDRESS_2,CITY,REGION,COUNTRY_CODE,SEGMENT_NAME,SEGMENT_DESCRIPTION,RETAILER_TYPE_EN,CONTACT_FIRST_NAME,CONTACT_LAST_NAME,CONTACT_E_MAIL,CONTACT_JOB_POSITION_EN
0,93.0,14.0,Le Golfeur,Le Golfeur,"500, Place d'Armes",Bureau 1061,Montréal,Québec,4.0,Golf Shop,Primarily offers golf equipment and accessories.,Golf Shop,Frank,Smith,FSmith@legolfeurinc.com,Chief Purchaser
2,49.0,42.0,Holstein Golf,Holstein Golf,Jubelstraße 23,None,Berlin,None,2.0,Golf Shop,Primarily offers golf equipment and accessories.,Golf Shop,Klaus,Bülow,KBulow@holsteing.com,Site Assistant Manager
12,100.0,73.0,Golf Masters,Golf Masters,2845 South Second Street,None,Lincoln,Nebraska,3.0,Golf Shop,Primarily offers golf equipment and accessories.,Golf Shop,Edward,Hurley,Ed@ulyssesnet5.net,Site Manager
16,110.0,109.0,The Golf Hut,The Golf Hut,1488 Orleans Road,P.O. Box 1128,Charleston,South Carolina,3.0,Golf Shop,Primarily offers golf equipment and accessories.,Golf Shop,Stephanie,Kosawski,SK@ghni7.com,Site Manager
20,121.0,151.0,The Caddy's Corner,The Caddy's Corner,1529 Kapahula Avenue,None,Honolulu,Hawaii,3.0,Golf Shop,Primarily offers golf equipment and accessories.,Golf Shop,Ryan,Livingston,Livingston@cc4golf.com,Assistant Purchaser
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,146.0,289.0,Extra Sport,Extra Sport,Stadhouderslaan 18,None,Rotterdam,None,7.0,Sports Store - Exclusive,Primarily offers a wide range of sporting good...,Sports Store,Bill,Baas,BBaas@exsp2k.com,Site Manager
746,147.0,304.0,NonSoloNeve,NonSoloNeve,Corso XXII Marzo 69,None,Bologna,None,6.0,Sports Store - Exclusive,Primarily offers a wide range of sporting good...,Sports Store,Alberto,Cigli,ACigli@nonsoloneve.com,Site Assistant Manager
756,171.0,352.0,Taeho Sports,Taeho Sports,"56-3, Namchun-Dong",Suyoung-Ku,Pusan,None,13.0,Sports Store,Primarily offers a wide range of sporting goods.,Sports Store,Mi-na,Lee,MLee@taehosports.com,Site Manager
766,178.0,393.0,Beck's Sports Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sports Store,Joanne,Wallen,JWallen@beckssportsstore.com,Chief Purchaser


In [22]:
def upload_dataframe(df, table_name):
    print(f"Uploading {table_name}...")
    df.to_sql(table_name, wk3_engine, if_exists='replace', index=False)
    print(f"{table_name} uploaded successfully.")

dimension_tables = {
    'fact_omzetten': fact_omzetten,
}

for table_name, df in dimension_tables.items():
    upload_dataframe(df, table_name)


print("Alle data zijn geupload naar de database.")

Uploading fact_omzetten...
fact_omzetten uploaded successfully.
Alle data zijn geupload naar de database.


## dimensie tabel4 dim_sales

In [18]:
# Maak een kopie van de sales_staff DataFrame
dim_sales_df = df_sales_staff.copy()

dim_sales_df = dim_sales_df.drop_duplicates(subset=['SALES_STAFF_CODE'])
dim_sales_df = dim_sales_df.dropna(subset=['SALES_STAFF_CODE'])
dim_sales_df['DATE_HIRED'] = pd.to_datetime(dim_sales_df['DATE_HIRED']).dt.strftime('%Y%m%d').astype(int)

dim_sales_df


,FAX,POSITION_EN,WORK_PHONE,SALES_STAFF_CODE,LAST_NAME,EMAIL,DATE_HIRED,FIRST_NAME,SALES_BRANCH_CODE,MANAGER_CODE,EXTENSION
0,+33 1 68 94 56 60,Branch Manager,+33 1 68 94 52 20,4,Pagé,DPage@grtd123.com,19961103,Denis,6,NaN,325.0
1,+33 1 68 94 56 60,Level 3 Sales Representative,+33 1 68 94 52 20,5,Michel,EMichel@grtd123.com,19950608,Élizabeth,6,NaN,336.0
2,+33 1 68 94 56 60,Level 1 Sales Representative,+33 1 68 94 52 20,6,Clermont,EClermont@grtd123.com,19980407,Émile,6,NaN,378.0
3,+33 1 68 94 56 60,Level 2 Sales Representative,+33 1 68 94 52 20,7,Jauvin,EJauvin@grtd123.com,19970816,Étienne,6,NaN,398.0
4,+(49) 40 663 4571,Level 2 Sales Representative,+(49) 40 663 1990,12,Wiesinger,EWiesinger@grtd123.com,19970227,Elsbeth,13,NaN,1818.0
...,...,...,...,...,...,...,...,...,...,...,...
97,+32 16 20.73.32,Level 1 Sales Representative,+32 16 20.73.21,120,Laermans,GLaermans@grtd123.com,19991115,Giele,38,NaN,1340.0
98,+32 16 20.73.32,Level 1 Sales Representative,+32 16 20.73.21,121,De Crée,FDecree@grtd123.com,19991201,François,38,NaN,1642.0
99,+32 16 20.73.32,Level 3 Sales Representative,+32 16 20.73.21,122,Lattrez,YLattrez@grtd123.com,19991209,Yvette,38,NaN,1633.0
100,+(43) 13 79 56 33,Level 2 Sales Representative,+(43) 13 79 56 32,123,Seefelder,WSeefelder@grtd123.com,19981028,Willi,39,NaN,325.0


## dimensie tabel5 dim_sales_branch

In [19]:
dim_sales_branch_df = df_sales_branch.copy()
dim_sales_branch_df = dim_sales_branch_df.rename(columns={
    'ADDRESS1': 'ADDRESS_1',
    'ADDRESS2': 'ADDRESS_2'
})
dim_sales_branch_df = dim_sales_branch_df[['SALES_BRANCH_CODE', 'ADDRESS_1', 'ADDRESS_2', 'CITY', 'REGION', 'COUNTRY_CODE']]
dim_sales_branch_df['COUNTRY_CODE'] = dim_sales_branch_df['COUNTRY_CODE'].astype(int)  # Converteer naar string als nodig


dim_sales_branch_df


,SALES_BRANCH_CODE,ADDRESS_1,ADDRESS_2,CITY,REGION,COUNTRY_CODE
0,6,"75, rue du Faubourg St-Honoré",None,Paris,None,1
1,7,"Piazza Duomo, 1",None,Milano,None,6
2,9,Singelgravenplein 4,4e verdieping,Amsterdam,Noord-Holland,7
3,13,Schwabentor 35,None,Hamburg,None,2
4,14,Leopoldstraße 36,None,München,None,2
5,15,Isafjordsgatan 30 C,None,Kista,None,10
6,17,"7800, 756 - 6th Avenue. S.W.",None,Calgary,Alberta,4
7,18,789 Yonge Street,None,Toronto,Ontario,4
8,19,1288 Dorchester Avenue,None,Boston,Massachusetts,3
9,20,299 Yale Avenue,None,Seattle,Washington,3


In [20]:
df_order_header

,RETAILER_CONTACT_CODE,RETAILER_NAME,ORDER_DATE,ORDER_METHOD_CODE,RETAILER_SITE_CODE,SALES_STAFF_CODE,ORDER_NUMBER,SALES_BRANCH_CODE
0,6,Ultra Sports,2022-04-14,7,4,50,1153,18
1,6,Ultra Sports,2022-10-01,4,4,50,1154,18
2,65,Ultra Sports,2022-04-21,7,5,49,1155,18
3,65,Ultra Sports,2022-09-09,5,5,49,1156,18
4,66,Ultra Sports,2022-04-10,2,6,50,1157,18
...,...,...,...,...,...,...,...,...
4779,329,Preben's T°y,2022-01-13,4,338,29,9475,15
4780,327,Preben's T°y,2023-01-02,5,336,29,9476,15
4781,329,Preben's T°y,2023-01-18,2,338,29,9477,15
4782,327,Preben's T°y,2024-01-15,7,336,27,9478,15


## feiten tabel- fact_omzetten

In [21]:
# Stap 1: Merge order_header en order_details op ORDER_NUMBER
fact_omzetten = pd.merge(df_order_header, df_order_details, on="ORDER_NUMBER", how="inner")

# Stap 2: Voeg ORDER_METHOD_EN toe via een merge met df_order_method
fact_omzetten = pd.merge(fact_omzetten,
                         df_order_method[['ORDER_METHOD_CODE', 'ORDER_METHOD_EN']],
                         on="ORDER_METHOD_CODE", how="left")

# Stap 3: Selecteer de benodigde kolommen
fact_omzetten = fact_omzetten[['ORDER_NUMBER',
                               'RETAILER_SITE_CODE',
                               'RETAILER_CONTACT_CODE',
                               'SALES_STAFF_CODE',
                               'SALES_BRANCH_CODE',
                               'ORDER_DATE',
                               'ORDER_METHOD_EN',
                               'ORDER_DETAIL_CODE',
                               'PRODUCT_NUMBER',
                               'QUANTITY',
                               'UNIT_PRICE',
                               'UNIT_COST',
                               'UNIT_SALE_PRICE']]

# Converteer de ORDER_DATE kolom van string naar datetime en dan naar een int in het formaat YYYYMMDD
fact_omzetten['ORDER_DATE'] = pd.to_datetime(fact_omzetten['ORDER_DATE']).dt.strftime('%Y%m%d').astype(int)

# Stap 4: Bereken de afgeleide meetwaarden en rond af op twee decimalen
fact_omzetten['TOTAL_REVENUE'] = (fact_omzetten['QUANTITY'] * fact_omzetten['UNIT_SALE_PRICE']).round(2)
fact_omzetten['TOTAL_COST']    = (fact_omzetten['QUANTITY'] * fact_omzetten['UNIT_COST']).round(2)
fact_omzetten['PROFIT']        = (fact_omzetten['TOTAL_REVENUE'] - fact_omzetten['TOTAL_COST']).round(2)

fact_omzetten = fact_omzetten.drop_duplicates(subset=['RETAILER_SITE_CODE'])
fact_omzetten = fact_omzetten.dropna(subset=['RETAILER_SITE_CODE'])

# Bekijk de resulterende DataFrame
fact_omzetten

,ORDER_NUMBER,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,SALES_STAFF_CODE,SALES_BRANCH_CODE,ORDER_DATE,ORDER_METHOD_EN,ORDER_DETAIL_CODE,PRODUCT_NUMBER,QUANTITY,UNIT_PRICE,UNIT_COST,UNIT_SALE_PRICE,TOTAL_REVENUE,TOTAL_COST,PROFIT
0,1153,4,6,50,18,20220414,Sales visit,47747,2,100,12.53,9.64,11.81,1181.00,964.00,217.00
10,1155,5,65,49,18,20220421,Sales visit,47753,2,58,12.53,9.64,12.53,726.74,559.12,167.62
22,1157,6,66,50,18,20220410,Telephone,47799,8,44,151.77,88.23,135.88,5978.72,3882.12,2096.60
35,1159,9,68,50,18,20220405,Sales visit,47738,1,126,6.59,4.38,6.59,830.34,551.88,278.46
42,1161,13,70,49,18,20220327,Fax,47742,1,232,6.59,4.38,5.42,1257.44,1016.16,241.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36646,9396,340,331,29,15,20220406,Web,109733,2,38,12.53,9.64,12.53,476.14,366.32,109.82
36656,9399,341,332,29,15,20220411,Telephone,109813,88,160,6.63,1.88,6.63,1060.80,300.80,760.00
36662,9402,336,327,29,15,20220312,Web,109737,5,18,54.93,38.40,54.93,988.74,691.20,297.54
36675,9405,338,329,28,15,20220329,Web,109732,1,34,6.59,4.38,6.59,224.06,148.92,75.14


## feiten tabel2 - fact_retouren

In [14]:
# Stap 1: Merge returned_item met order_details op ORDER_DETAIL_CODE om PRODUCT_NUMBER te krijgen
fact_retouren = pd.merge(df_returned_item,
                         df_order_details[['ORDER_DETAIL_CODE', 'PRODUCT_NUMBER']],
                         on='ORDER_DETAIL_CODE', how='left')

# Stap 2: Merge met return_reason om de beschrijving voor RETURN_REASON te krijgen
fact_retouren = pd.merge(fact_retouren,
                         df_return_reason[['RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']],
                         on='RETURN_REASON_CODE', how='left')

# Stap 3: Selecteer de gewenste kolommen en hernoem indien nodig
fact_retouren = fact_retouren[['RETURN_DATE', 'ORDER_DETAIL_CODE', 'RETURN_DESCRIPTION_EN', 'RETURN_QUANTITY', 'PRODUCT_NUMBER']]
fact_retouren.rename(columns={'ORDER_DETAIL_CODE': 'ORDER_DETAIL',
                              'RETURN_DESCRIPTION_EN': 'RETURN_REASON'}, inplace=True)

# Converteer de RETURN_DATE kolom van bijvoorbeeld "Aug 1 2023 4:10AM" naar een datetime,
# en formatteer deze als een integer in het formaat YYYYMMDD.
fact_retouren['RETURN_DATE'] = pd.to_datetime(fact_retouren['RETURN_DATE']).dt.strftime('%Y%m%d').astype(int)

# Bekijk het resultaat
fact_retouren.head()


,RETURN_DATE,ORDER_DETAIL,RETURN_REASON,RETURN_QUANTITY,PRODUCT_NUMBER
0,20230801,84858,Unsatisfactory product,8,6
1,20221206,84440,Incomplete product,2,8
2,20230624,84867,Wrong product shipped,22,8
3,20230721,84873,Wrong product ordered,20,8
4,20221007,84488,Defective product,2,17


# feiten tabel3 - fact_voorraadaantallen

In [21]:
# Maak een kopie van de inventory DataFrame
fact_voorraadaantallen = df_inventory.copy()

# Creëer een nieuwe DATE kolom in formaat YYYYMMDD (dag = 1)
fact_voorraadaantallen['DATE'] = (fact_voorraadaantallen['INVENTORY_YEAR'] * 10000 +
                                    fact_voorraadaantallen['INVENTORY_MONTH'] * 100 + 1)

# Selecteer de benodigde kolommen voor fact_voorraadaantallen
fact_voorraadaantallen = fact_voorraadaantallen[['DATE', 'PRODUCT_NUMBER', 'INVENTORY_COUNT']]


fact_voorraadaantallen


,DATE,PRODUCT_NUMBER,INVENTORY_COUNT
0,20230401,48,1932
1,20230401,49,1400
2,20230401,50,21705
3,20230401,51,9710
4,20230401,52,5616
...,...,...,...
3538,20230401,43,6147
3539,20230401,44,5822
3540,20230401,45,1676
3541,20230401,46,1802


## feiten tabel4 fact_verkoopverwachtingen

In [23]:
fact_verkoopverwachtingen = df_product_forecast.copy()

# Creëer een DATE-kolom op basis van YEAR en MONTH; stel de dag standaard op 01.
fact_verkoopverwachtingen['FORECAST_DATE'] = fact_verkoopverwachtingen['YEAR'] * 10000 + fact_verkoopverwachtingen['MONTH'] * 100 + 1

# Selecteer de gewenste kolommen
fact_verkoopverwachtingen = fact_verkoopverwachtingen[['PRODUCT_NUMBER', 'FORECAST_DATE', 'EXPECTED_VOLUME']]

fact_verkoopverwachtingen

,PRODUCT_NUMBER,FORECAST_DATE,EXPECTED_VOLUME
0,45,20231201,90
1,46,20231201,84
2,47,20231201,468
3,48,20231201,91
4,49,20231201,35
...,...,...,...
3524,37,20220101,60
3525,38,20220101,51
3526,39,20220101,235
3527,40,20220101,568


## feiten tabel5 fact_klant_tevredenheid

In [17]:
fact_klant_tevredenheid = df_satisfaction.copy()

# Maak een DATE-kolom op basis van YEAR (stel dag op 01)
fact_klant_tevredenheid['DATE'] = fact_klant_tevredenheid['YEAR'] * 10000 + 101

# Nu mergen we met df_satisfaction_type om de tekstuele beschrijving te verkrijgen.
# We gaan ervan uit dat df_satisfaction_type de kolommen bevat: SATISFACTION_TYPE_CODE en SATISFACTION_TYPE_DESCRIPTION.
fact_klant_tevredenheid = fact_klant_tevredenheid.merge(
    df_satisfaction_type[['SATISFACTION_TYPE_CODE', 'SATISFACTION_TYPE_DESCRIPTION']],
    on='SATISFACTION_TYPE_CODE',
    how='left'
)

# Hernoem SATISFACTION_TYPE_DESCRIPTION naar SATISFACTION_TYPE
fact_klant_tevredenheid = fact_klant_tevredenheid.rename(
    columns={'SATISFACTION_TYPE_DESCRIPTION': 'SATISFACTION_TYPE'}
)

# Selecteer de benodigde kolommen
fact_klant_tevredenheid = fact_klant_tevredenheid[['DATE', 'SALES_STAFF_CODE', 'SATISFACTION_TYPE']]

fact_klant_tevredenheid

,DATE,SALES_STAFF_CODE,SATISFACTION_TYPE
0,20220101,4,More than satisfied
1,20220101,5,Very Satisfied
2,20220101,6,Satisfied
3,20220101,7,Very Satisfied
4,20220101,12,Very Satisfied
...,...,...,...
197,20230101,120,Very Satisfied
198,20230101,121,Satisfied
199,20230101,122,Very Satisfied
200,20230101,123,Satisfied


## feiten tabel6 fact_trainingen

In [18]:
fact_trainingen = pd.merge(df_training, df_course[['COURSE_CODE', 'COURSE_DESCRIPTION']], on='COURSE_CODE', how='left')

# DATE kolom op basis van YEAR
fact_trainingen['DATE'] = fact_trainingen['YEAR'] * 10000 + 101

# Selecteer de benodigde kolommen
fact_trainingen = fact_trainingen[['SALES_STAFF_CODE', 'COURSE_DESCRIPTION', 'DATE']]

fact_trainingen

,SALES_STAFF_CODE,COURSE_DESCRIPTION,DATE
0,4,GO Marketing 3,20220101
1,5,GO Marketing 1,20220101
2,5,GO Marketing 2,20220101
3,5,GO Marketing 3,20220101
4,6,GO Sales 1,20220101
...,...,...,...
314,121,GO Marketing 1,20230101
315,122,GO Marketing 2,20230101
316,123,GO Marketing 3,20230101
317,124,GO Marketing 2,20230101


## dim_time tabel moest als laatste gedaan worden en als eerste in database gezet worden

In [34]:
# Collect all unique date values from the relevant fact tables
dates_omzetten = fact_omzetten['ORDER_DATE'].drop_duplicates()
dates_trainingen = fact_trainingen['DATE'].drop_duplicates()
dates_klanttevredenheid = fact_klant_tevredenheid['DATE'].drop_duplicates()
dates_retouren = fact_retouren['RETURN_DATE'].drop_duplicates()
dates_voorraad = fact_voorraadaantallen['DATE'].drop_duplicates()
dates_forecast = fact_verkoopverwachtingen['FORECAST_DATE'].drop_duplicates()

# Combine all date ranges
all_dates = pd.concat([
    dates_omzetten,
    dates_trainingen,
    dates_klanttevredenheid,
    dates_retouren,
    dates_voorraad,
    dates_forecast
]).drop_duplicates()

# Convert to datetime
all_dates = pd.to_datetime(all_dates.astype(str), errors='coerce').dropna().sort_values()

# Build the dim_time DataFrame with only DATE, YEAR, MONTH, DAY
dim_time_df = pd.DataFrame({'DATE': all_dates})
dim_time_df['YEAR'] = dim_time_df['DATE'].dt.year
dim_time_df['MONTH'] = dim_time_df['DATE'].dt.month
dim_time_df['DAY'] = dim_time_df['DATE'].dt.day

# Converteer de DATE kolom naar een string in 'YYYYMMDD' en daarna naar een integer
dim_time_df['DATE'] = dim_time_df['DATE'].dt.strftime('%Y%m%d').astype(int)

# Selecteer de gewenste kolommen (als de volgorde belangrijk is)
dim_time_df = dim_time_df[['DATE', 'YEAR', 'MONTH', 'DAY']]

dim_time_df


,DATE,YEAR,MONTH,DAY
0,20220101,2022,1,1
2769,20220102,2022,1,2
13666,20220103,2022,1,3
2742,20220105,2022,1,5
2760,20220107,2022,1,7
...,...,...,...,...
193,20241024,2024,10,24
487,20241026,2024,10,26
201,20241110,2024,11,10
594,20241114,2024,11,14


# Alle dataframes naar database sturen

In [91]:
def upload_dataframe(df, table_name):
    print(f"Uploading {table_name}...")
    df.to_sql(table_name, wk3_engine, if_exists='replace', index=False)
    print(f"{table_name} uploaded successfully.")

# Lijst van dimensietabellen
dimension_tables = {
    'dim_time': dim_time_df,
    'dim_retailer': dim_retailer_df,
    'dim_product': dim_product_df,
    'dim_country': dim_country_df,
    'dim_sales_branch': dim_sales_branch_df,
    'dim_sales': dim_sales_df
}

# Lijst van feitentabellen
fact_tables = {
    'fact_omzetten': fact_omzetten,
    'fact_retouren': fact_retouren,
    'fact_voorraadaantallen': fact_voorraadaantallen,
    'fact_verkoopverwachtingen': fact_verkoopverwachtingen,
    'fact_trainingen': fact_trainingen,
    'fact_klant_tevredenheid': fact_klant_tevredenheid
}

# Eerst dimensietabellen uploaden
for table_name, df in dimension_tables.items():
    upload_dataframe(df, table_name)

# Daarna feitentabellen uploaden
for table_name, df in fact_tables.items():
    upload_dataframe(df, table_name)

print("Alle tabellen zijn geupload naar de database.")


Uploading dim_time...
dim_time uploaded successfully.
Uploading dim_retailer...
dim_retailer uploaded successfully.
Uploading dim_product...
dim_product uploaded successfully.
Uploading dim_country...
dim_country uploaded successfully.
Uploading dim_sales_branch...
dim_sales_branch uploaded successfully.
Uploading dim_sales...
dim_sales uploaded successfully.
Uploading fact_omzetten...
fact_omzetten uploaded successfully.
Uploading fact_retouren...
fact_retouren uploaded successfully.
Uploading fact_voorraadaantallen...
fact_voorraadaantallen uploaded successfully.
Uploading fact_verkoopverwachtingen...
fact_verkoopverwachtingen uploaded successfully.
Uploading fact_trainingen...
fact_trainingen uploaded successfully.
Uploading fact_klant_tevredenheid...
fact_klant_tevredenheid uploaded successfully.
Alle tabellen zijn geupload naar de database.
